In [2]:
from utils import *

provider_path = join(home, 'IRCCS-CRC')
update_path = "/Users/tushar/CancerModels/submission/IRCCS-CRC"

In [3]:
file_dict = {'pdx': 'IRCCS-CRC_metadata-pdx_model.tsv', 'mv': 'IRCCS-CRC_metadata-model_validation.tsv', 'ps': 'IRCCS-CRC_metadata-patient_sample.tsv', 'p': 'IRCCS-CRC_metadata-patient.tsv', 'share': 'IRCCS-CRC_metadata-sharing.tsv'}

In [4]:
temp = read_metadata_with_fields(join(provider_path, file_dict['ps']))
clinical = pd.read_excel(join(update_path, '41467_2024_51909_MOESM4_ESM.xlsx'), sheet_name='Supplementary Data 1', skiprows=[0, 1])
present_in_cm = dict()
for c in clinical.CASE:
    for m in temp.model_id:
        if m.__contains__(c):
            present_in_cm[c] =  m

In [5]:
temp[[ True if p in clinical['Patient ID'] else False for p in temp.patient_id]]

,Field,patient_id,sample_id,collection_date,collection_event,months_since_collection_1,age_in_years_at_collection,diagnosis,tumour_type,primary_site,...,grade,grading_system,virology_status,gene_mutation_status,sharable,treatment_naive_at_collection,treated_at_collection,treated_prior_to_collection,response_to_treatment,model_id


In [6]:
for _, row in clinical.iterrows():
    if row['CASE'] not in present_in_cm.keys():
        continue
    ps = temp[temp['model_id'] == present_in_cm[row['CASE']] ].reset_index(drop=True)
    location = temp[temp['model_id'] == present_in_cm[row['CASE']] ].index[0]
    tnac = 'Yes' if row[' THERAPY BEFORE COLLECTION (Y/N)']  == 'Y' else 'No'
    tpc = tnac
    tac = 'No' if tnac == "Yes" else 'No'
    cdate = row['PRIMARY DIAGNOSIS']
    if str(cdate) == 'NaT':
        cdate = 'Not provided'
    else:
        cdate = pd.to_datetime(cdate, format='%Y%m%d %H:%M:%S').strftime('%b-%Y')
    temp.at[location, 'treatment_naive_at_collection'] = tnac
    temp.at[location, 'treated_prior_to_collection'] = tpc
    temp.at[location, 'treated_at_collection'] = tac
    temp.at[location, 'collection_date'] = cdate

In [7]:
to_add_ps = pd.DataFrame()
for _, row in clinical.iterrows():
    if row['CASE'] in present_in_cm.keys():
        continue
    cdate = row['PRIMARY DIAGNOSIS']
    if str(cdate) == 'NaT':
        cdate = 'Not provided'
    else:
        cdate = pd.to_datetime(cdate, format='%Y%m%d %H:%M:%S').strftime('%b-%Y')
    stage = row['STAGE']
    if str(stage) == 'nan':
        stage = 'Not provided'
    else:
        stage = int(stage)
    tnac = 'Yes' if row[' THERAPY BEFORE COLLECTION (Y/N)']  == 'Y' else 'No'
    tpc = tnac
    tac = 'No' if tnac == "Yes" else 'No'
    pid = int(row['Patient ID']) if str(row['Patient ID']) != 'nan' else "XP_" + row['CASE']
    add_row = ['', pid, row['CASE'], cdate, '', '', row['AGE AT COLLECTION (≤64; 65-74: ≥75)'], 'Colorectal Carcinoma', 'Metastatic', row['SITE OF PRIMARY'], 'Liver', 'metastasectomy', stage, '', 'Not provided', 'Not provided', 'Not provided', 'Not provided', 'Yes', tnac, tpc, tac, '', 'XENTURION_'+row['CASE']]
    add_row = pd.DataFrame([add_row], columns=temp.columns)
    to_add_ps = pd.concat([to_add_ps, add_row]).reset_index(drop=True)

In [23]:
subset = clinical[ [True if pid in temp['patient_id'].astype(str).unique() else False for pid in clinical['Patient ID'].astype(str)]]
subset = subset[['CASE', 'MSI/MSS']]
subset['model_id'] = 'XENTURION_'+subset['CASE']
subset['sample_id'] = subset['CASE']

In [25]:
subset.to_csv(join(provider_path, 'IRCCS-CRC_immunemarker-Sheet1.tsv'), sep='\t', index=False)

In [15]:
temp.patient_id

0      Unique anonymous/de-identified provider ID for...
1                                                  A0088
2                           free alphanumerical and -_~.
3                                              essential
4                                               11000206
                             ...                        
699                                             11006180
700                                             21001315
701                                             21001331
702                                             11006427
703                                             11006326
Name: patient_id, Length: 704, dtype: object

In [127]:
pd.concat([temp, to_add_ps]).to_csv(join(provider_path, file_dict['ps']), sep='\t', index=False)

In [142]:
p_df = read_metadata_with_fields(join(provider_path, file_dict['p']))
to_add_p = pd.DataFrame()
for p in to_add_ps.patient_id.unique():
    sub = clinical[clinical['Patient ID'] == p].reset_index(drop=True)
    sub['patient_id'] = sub['Patient ID']
    sub['sex'] = sub['SEX']
    sub['sex'] = ['Female' if s == 'F' else 'Male' for s in sub['sex']]
    sub['initial_diagnosis'] = 'Colorectal Carcinoma'
    sub['age_at_initial_diagnosis'] = sub['AGE AT COLLECTION (≤64; 65-74: ≥75)']
    sub['history'] = sub['DEAD/ALIVE']
    for col in p_df.columns:
        if col not in sub.columns:
            sub[col] = 'Not provided'
    to_add_p = pd.concat([to_add_p, sub[p_df.columns]])
to_add_p

,Field,patient_id,sex,history,ethnicity,ethnicity_assessment_method,initial_diagnosis,age_at_initial_diagnosis,age_category,smoking_status,alcohol_status,alcohol_frequency,family_history_of_cancer
0,Not provided,11003176,Female,NaN,Not provided,Not provided,Colorectal Carcinoma,≤64,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001036,Male,DEAD,Not provided,Not provided,Colorectal Carcinoma,≥75,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001049,Male,DEAD,Not provided,Not provided,Colorectal Carcinoma,≤64,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001043,Male,ALIVE,Not provided,Not provided,Colorectal Carcinoma,65-74,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001041,Female,DEAD,Not provided,Not provided,Colorectal Carcinoma,65-74,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001053,Male,DEAD,Not provided,Not provided,Colorectal Carcinoma,65-74,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001054,Female,DEAD,Not provided,Not provided,Colorectal Carcinoma,≤64,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001058,Female,ALIVE,Not provided,Not provided,Colorectal Carcinoma,≤64,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001062,Male,DEAD,Not provided,Not provided,Colorectal Carcinoma,≥75,Not provided,Not provided,Not provided,Not provided,Not provided
0,Not provided,21001060,Female,ALIVE,Not provided,Not provided,Colorectal Carcinoma,≤64,Not provided,Not provided,Not provided,Not provided,Not provided


In [143]:
pd.concat([p_df, to_add_p]).to_csv(join(provider_path, file_dict['p']), sep='\t', index=False)

In [83]:
temp.to_csv(join(provider_path, file_dict['ps']), sep='\t', index=False)

In [87]:
new_model = [c for c in clinical.CASE if c not in present_in_cm.keys()] 

In [138]:
clinical

,CASE,Patient ID,HOSPITAL,COLLECTION DATE (yyy-mm-dd),AGE AT COLLECTION (≤64; 65-74: ≥75),SEX,SITE OF PRIMARY,STAGE,T,N,...,THERAPY BEFORE COLLECTION (Y/N),THERAPY,CETUXIMAB THERAPY (Y/N),RESPONSE TO CETUXIMAB THERAPY,RELAPSE AFTER COLLECTION,FOLLOW UP (YEAR),DFS (Y/N),DEAD/ALIVE,PRIMARY TUMOR COLLECTED (Y/N),MSI/MSS
0,CRC0014,200800005,Candiolo Cancer Institute,2008-10-01,65-74,M,RECTUM,2.0,3.0,0.0,...,Y,FOLFIRI,N,N,N,2019.0,NaN,NaN,N,MSS
1,CRC0018,200800010,Candiolo Cancer Institute,2008-10-24,≤64,F,SIGMOID COLON,4.0,4.0,0.0,...,Y,FOLFIRI+BEVACIZUMAB,N,N,Y,2011.0,Y,DEAD,N,MSS
2,CRC0019,200900026,Candiolo Cancer Institute,2009-03-23,≥75,M,RIGHT COLON,4.0,3.0,2.0,...,N,N,N,N,Y,2012.0,Y,DEAD,N,MSS
3,CRC0022,200800021,Candiolo Cancer Institute,2008-12-17,65-74,M,SIGMOID COLON,3.0,3.0,1.0,...,Y,FOLFIRI+BEVACIZUMAB,N,N,Y,2010.0,Y,DEAD,N,MSS
4,CRC0025,200900002,Candiolo Cancer Institute,2009-01-12,≤64,M,HEPATIC FLEXURE,4.0,3.0,1.0,...,Y,FOLFOX,N,N,N,2017.0,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,CRC2573,11006180,Candiolo Cancer Institute,2020-07-30,65-74,M,LEFT COLON,4.0,4.0,1.0,...,Y,FOLFOX,N,N,N,2020.0,NaN,NaN,N,MSS
263,CRC2597,21001315,Mauriziano Umberto I Hospital,2020-08-31,65-74,M,RECTUM,4.0,3.0,1.0,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
264,CRC2606,21001331,Mauriziano Umberto I Hospital,2020-09-14,≤64,F,RECTUM,3.0,4.0,2.0,...,Y,XELOX,N,N,N,2020.0,N,ALIVE,N,NaN
265,CRC2772,11006427,Candiolo Cancer Institute,2021-01-18,65-74,M,LEFT COLON,4.0,3.0,0.0,...,Y,FOLFOX+BEVACIZUMAB,N,N,N,NaN,NaN,NaN,N,MSS


In [104]:
patient_in_cm = dict()
cm_patient_id = temp.iloc[4:].patient_id.unique()
for c in clinical['Patient ID']:
        if str(c) in cm_patient_id:
            patient_in_cm[c] = True

In [107]:
patient_in_cm

{200800002: True,
 200800007: True,
 200900004: True,
 200900028: True,
 200900029: True,
 200900030: True,
 200900040: True,
 200900043: True,
 200900017: True,
 200900061: True,
 201000016: True,
 201100006: True,
 200900045: True,
 200900073: True,
 201100075: True,
 201100176: True,
 'MOL003': True,
 'MOL004': True,
 31001003: True,
 21000638: True,
 21000642: True,
 31001014: True,
 21000697: True,
 11002234: True,
 21000788: True,
 11002372: True,
 21000727: True,
 21000792: True,
 31001024: True,
 21000816: True,
 21000819: True,
 21000826: True,
 21000832: True,
 21000834: True,
 21000839: True,
 21000843: True,
 11002719: True,
 11002828: True,
 11002779: True,
 11002732: True,
 11002755: True,
 11002776: True,
 11002925: True,
 21000857: True,
 21000859: True,
 21000863: True,
 21000739: True,
 21000735: True,
 21000770: True,
 21000767: True,
 21000748: True,
 21000756: True,
 21000757: True,
 21000734: True,
 11003262: True,
 21000877: True,
 21000878: True,
 21000730: True

In [96]:
cm_patiend_id

array(['11000206', '11000379', '11000380', '11000481', '11000497',
       '11001516', '11001636', '11001638', '11001778', '11001824',
       '11001850', '11001863', '11001874', '11001908', '11001917',
       '11001922', '11001925', '11001935', '11001944', '11001946',
       '11002023', '11002026', '11002218', '11002222', '11002223',
       '11002230', '11002234', '11002239', '11002244', '11002260',
       '11002305', '11002306', '11002332', '11002353', '11002354',
       '11002367', '11002372', '11002419', '11002518', '11002529',
       '11002557', '11002558', '11002561', '11002564', '11002600',
       '11002605', '11002607', '11002642', '11002647', '11002666',
       '11002704', '11002717', '11002719', '11002729', '11002731',
       '11002732', '11002733', '11002738', '11002746', '11002751',
       '11002755', '11002761', '11002764', '11002776', '11002779',
       '11002780', '11002783', '11002795', '11002808', '11002812',
       '11002813', '11002818', '11002820', '11002825', '110028

In [108]:
present_in_cm

{'CRC0014': 'CRC0014LM',
 'CRC0018': 'CRC0018LM',
 'CRC0019': 'CRC0019LM',
 'CRC0022': 'CRC0022LM',
 'CRC0025': 'CRC0025LM',
 'CRC0028': 'CRC0028LM',
 'CRC0029': 'CRC0029LM',
 'CRC0030': 'CRC0030LM',
 'CRC0031': 'CRC0031LM',
 'CRC0051': 'CRC0051LM',
 'CRC0055': 'CRC0055LM',
 'CRC0058': 'CRC0058LM',
 'CRC0059': 'CRC0059LM',
 'CRC0060': 'CRC0060LM',
 'CRC0063': 'CRC0063LM',
 'CRC0064': 'CRC0064LM',
 'CRC0065': 'CRC0065PR',
 'CRC0066': 'CRC0066LM',
 'CRC0067': 'CRC0067LM',
 'CRC0068': 'CRC0068LM',
 'CRC0069': 'CRC0069LM',
 'CRC0072': 'CRC0072LM',
 'CRC0076': 'CRC0076LM',
 'CRC0077': 'CRC0077LM',
 'CRC0078': 'CRC0078LM',
 'CRC0079': 'CRC0079LM',
 'CRC0080': 'CRC0080LM',
 'CRC0081': 'CRC0081LM',
 'CRC0095': 'CRC0095LM',
 'CRC0096': 'CRC0096LM',
 'CRC0097': 'CRC0097LM',
 'CRC0099': 'CRC0099LM',
 'CRC0103': 'CRC0103LM',
 'CRC0104': 'CRC0104LM',
 'CRC0106': 'CRC0106LM',
 'CRC0110': 'CRC0110LM',
 'CRC0114': 'CRC0114LM',
 'CRC0116': 'CRC0116LM',
 'CRC0118': 'CRC0118LM',
 'CRC0119': 'CRC0119LM',


In [117]:
[str(r) for r in clinical.STAGE.unique()]

['2.0', '4.0', '3.0', 'nan', '1.0']